# Sequential Pattern Mining
## PrefixSpan Algorithm
PrefixSpan extracts frequent sequences with depth-first search **(DFS)** by executing SDB projection operations **`recursively`**.

In [1]:
import pandas as pd
import numpy as np
import sys
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sshtunnel 
import psycopg2.extras
import csv
import re

### 1. Reading and Processing the Dataset

In [2]:
def get_data_regularity():
    
    #SQL statement regularity_trip_timegroups_results
    sql = """select *
from punctuality_new
where route_id=50 or route_id=15
order by route_short_name, stop_id, direction_id, timestamp"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [3]:
def get_defined_intervals():
    
    #SQL statement regularity_trip_timegroups_results
    sql = """select *
from defined_intervals
order by from_time"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [4]:
data=get_data_regularity()


In [5]:
data=data.dropna()
intervals=get_defined_intervals()
intervals["data"]=None
data["delay"]= data["delay"].astype(int)
data["stop_id"]= data["stop_id"].astype(str)



In [6]:
data["from_date_time"]= pd.to_datetime(data["timestamp"], unit='ms') + timedelta(hours=2)

In [7]:
data.head()

,id,route_short_name,route_id,route_type,direction_id,stop_id,date,time,timestamp,estimated_arrival,delay,closest_vpos,distance_from_point,from_date_time
0,2102168,50,50,3,0,1015,2021-09-06,13:36:00,1630928160000,1630928611738,451738,1630928661378,1,2021-09-06 13:36:00
1,2102169,50,50,3,0,1015,2021-09-06,13:49:00,1630928940000,1630929628088,688088,1630929628088,0,2021-09-06 13:49:00
2,2102170,50,50,3,0,1015,2021-09-06,14:01:00,1630929660000,1630929642265,0,1630929659804,76,2021-09-06 14:01:00
3,2102171,50,50,3,0,1015,2021-09-06,14:15:00,1630930500000,1630932331022,1831022,1630932356869,112,2021-09-06 14:15:00
4,2102172,50,50,3,0,1015,2021-09-06,14:29:00,1630931340000,1630932331022,991022,1630932356869,112,2021-09-06 14:29:00


In [8]:
data["inter"]=None
data["inter_day"]=None

data_final=[]
data_final_col=data.columns
for i in range(0, len(intervals)):
    c_interval=intervals.iloc[i].to_dict()
    temp=[]
    find_elements_range=data[(data["from_date_time"].dt.time>=c_interval["from_time"].time())&(c_interval["to_time"].time()>=data["from_date_time"].dt.time)&(data["from_date_time"].dt.date>=c_interval["from_time"].date())&(c_interval["to_time"].date()>=data["from_date_time"].dt.date)]
    for j in find_elements_range.to_dict(orient="records"):
        copy=j
        copy["inter_day"]=c_interval["from_time"].date().day
        copy["inter"]=c_interval["from_time"]
        data_final.append(copy)
        
data=pd.DataFrame(data_final, columns=data_final_col)

In [9]:
data=data[data["delay"]>(0)]

In [10]:
data_f=data[["route_id","stop_id","inter", "inter_day","delay"]]

In [11]:
data_f.to_csv("./data_final_puntuality.csv")